In [25]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import lightgbm as lgb
import joblib
import shap
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.cluster import MiniBatchKMeans
import warnings

warnings.filterwarnings("ignore")

RND = 42
np.random.seed(RND)

ROOT = Path("..")
DATA_DIR = ROOT / "data" / "processed"
OUT_DIR = ROOT / "models"
FIG_DIR = ROOT / "figures"
OUT_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

TRAIN_PARQUET = DATA_DIR / "train_with_label_1h.parquet"
TEST_PARQUET  = DATA_DIR / "test_with_label_1h.parquet"

In [26]:
train = pd.read_parquet(TRAIN_PARQUET)
test  = pd.read_parquet(TEST_PARQUET)

train = train.sort_values("ts").reset_index(drop=True)
test  = test.sort_values("ts").reset_index(drop=True)

print("Train rows:", len(train), "Test rows:", len(test))

Train rows: 1653203 Test rows: 318852


In [27]:
global_mean = train['label_1h'].mean()

grp = train.groupby('segment_id')['label_1h']
train['seg_count_so_far'] = grp.cumcount()
train['seg_sum_so_far']   = grp.cumsum() - train['label_1h']
train['seg_mean_so_far']  = train['seg_sum_so_far'] / train['seg_count_so_far'].replace(0, np.nan)

k = 10.0
train['seg_te_smoothed'] = (train['seg_mean_so_far'] * train['seg_count_so_far'] + global_mean * k) / (train['seg_count_so_far'] + k)
train['seg_te_smoothed'] = train['seg_te_smoothed'].fillna(global_mean)

agg = train.groupby('segment_id').agg(n=('label_1h','count'), s=('label_1h','sum'))
agg['mean'] = agg['s'] / agg['n']
agg['te_smoothed'] = (agg['mean'] * agg['n'] + global_mean * k) / (agg['n'] + k)
seg_te_map = agg['te_smoothed'].to_dict()

train['seg_te'] = train['seg_te_smoothed']
test['seg_te']  = test['segment_id'].map(seg_te_map).fillna(global_mean)

train = train.drop(columns=['seg_count_so_far','seg_sum_so_far','seg_mean_so_far','seg_te_smoothed'])
print("seg_te added. Global mean:", global_mean)

seg_te added. Global mean: 0.06642136507131913


In [28]:
def add_cyclical(df):
    df['hour'] = pd.to_datetime(df['ts']).dt.hour
    df['hour_sin'] = np.sin(2*np.pi*df['hour']/24)
    df['hour_cos'] = np.cos(2*np.pi*df['hour']/24)
    df['weekday'] = pd.to_datetime(df['ts']).dt.weekday
    df['weekday_sin'] = np.sin(2*np.pi*df['weekday']/7)
    df['weekday_cos'] = np.cos(2*np.pi*df['weekday']/7)
    return df

train = add_cyclical(train)
test  = add_cyclical(test)

In [29]:
DO_CLUSTER = True
K_CLUSTERS = 200

if DO_CLUSTER:
    coords = train[['LATITUDE','LONGITUDE']].dropna()
    kmeans = MiniBatchKMeans(n_clusters=K_CLUSTERS, random_state=RND, batch_size=10000)
    kmeans.fit(coords)
    train['cluster'] = kmeans.predict(train[['LATITUDE','LONGITUDE']].fillna(0))
    test['cluster']  = kmeans.predict(test[['LATITUDE','LONGITUDE']].fillna(0))

    cluster_agg = train.groupby('cluster')['label_1h'].agg(['count','mean'])
    cluster_global = train['label_1h'].mean()
    cluster_agg['te'] = (cluster_agg['mean']*cluster_agg['count'] + cluster_global*10.0) / (cluster_agg['count'] + 10.0)
    cluster_te_map = cluster_agg['te'].to_dict()
    train['cluster_te'] = train['cluster'].map(cluster_te_map).fillna(cluster_global)
    test['cluster_te']  = test['cluster'].map(cluster_te_map).fillna(cluster_global)
else:
    train['cluster_te'] = np.nan
    test['cluster_te']  = np.nan


In [30]:
TARGET = "label_1h"
DROP_COLS = ["ts","date","segment_id","BOROUGH","ZIP CODE","ON STREET NAME","CROSS STREET NAME","COLLISION_ID","window_start"]

# Prefer numeric columns + engineered ones
candidate_feats = [
    'hour_sin','hour_cos','weekday_sin','weekday_cos',
    'LATITUDE','LONGITUDE',
    'sev_1h','sev_6h','sev_24h','tot_1h','tot_6h','tot_24h',
    'seg_te','cluster_te'
]

# Filter only those present in the data
features = [f for f in candidate_feats if f in train.columns]
print("Using features:", features)

X_train = train[features].copy()
y_train = train[TARGET].astype(int).copy()
X_test  = test[features].copy()
y_test  = test[TARGET].astype(int).copy()

print("X_train shape:", X_train.shape, "X_test shape:", X_test.shape)

Using features: ['hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos', 'LATITUDE', 'LONGITUDE', 'sev_1h', 'sev_6h', 'sev_24h', 'tot_1h', 'tot_6h', 'tot_24h', 'seg_te', 'cluster_te']
X_train shape: (1653203, 14) X_test shape: (318852, 14)


In [19]:
tscv = TimeSeriesSplit(n_splits=5)
params = {
    "objective":"binary",
    "metric":"auc",
    "boosting_type":"gbdt",
    "seed":RND,
    "verbosity":-1,
    "learning_rate": 0.05,
    "num_leaves": 31,
    "max_depth": -1
}

aucs = []
fold = 0
for tr_idx, val_idx in tscv.split(X_train):
    X_tr, X_val = X_train.iloc[tr_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]
    dtr = lgb.Dataset(X_tr, label=y_tr)
    dval = lgb.Dataset(X_val, label=y_val, reference=dtr)
    bst = lgb.train(params, dtr, num_boost_round=500, valid_sets=[dval], callbacks=[lgb.callback.early_stopping(50), lgb.callback.log_evaluation(period=0)])
    yv = bst.predict(X_val, num_iteration=bst.best_iteration)
    auc_fold = roc_auc_score(y_val, yv)
    print(f"Fold {fold} AUC: {auc_fold:.4f}")
    aucs.append(auc_fold)
    fold += 1

print("Mean CV AUC:", np.mean(aucs))

Early stopping, best iteration is:
[78]	valid_0's auc: 0.967224
Fold 1 AUC: 0.9672
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[127]	valid_0's auc: 0.96739
Fold 2 AUC: 0.9674
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[109]	valid_0's auc: 0.966895
Fold 3 AUC: 0.9669
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[141]	valid_0's auc: 0.969157
Fold 4 AUC: 0.9692
Mean CV AUC: 0.9667615921553058


In [20]:
dtrain = lgb.Dataset(X_train, label=y_train)
dtest  = lgb.Dataset(X_test, label=y_test, reference=dtrain)

final_params = {**params}
bst_final = lgb.train(final_params, dtrain, num_boost_round=2000, valid_sets=[dtrain,dtest],
                      callbacks=[lgb.callback.early_stopping(50), lgb.callback.log_evaluation(period=50)])

best_iter = bst_final.best_iteration if getattr(bst_final, "best_iteration", None) else None
print("Best iteration:", best_iter)

Training until validation scores don't improve for 50 rounds
[50]	training's auc: 0.971194	valid_1's auc: 0.962197
[100]	training's auc: 0.97263	valid_1's auc: 0.963366
[150]	training's auc: 0.973646	valid_1's auc: 0.963353
Early stopping, best iteration is:
[133]	training's auc: 0.973309	valid_1's auc: 0.963454
Best iteration: 133


In [21]:
y_pred_prob = bst_final.predict(X_test, num_iteration=best_iter)
y_pred = (y_pred_prob >= 0.5).astype(int)

acc = accuracy_score(y_test, y_pred)
f1  = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred_prob)

print(f"Accuracy: {acc:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")

prevalence = y_test.mean()
k = 0.05
n_top = max(1, int(k * len(y_pred_prob)))
top_idx = np.argsort(y_pred_prob)[-n_top:]
model_prec_at_k = y_test.iloc[top_idx].mean()
print("Prevalence (test):", prevalence)
print(f"Model precision@{int(k*100)}%: {model_prec_at_k:.4f}")

Accuracy: 0.9513, F1: 0.4381, AUC: 0.9635
Prevalence (test): 0.048367267572416044
Model precision@5%: 0.4809


In [22]:
MODEL_PATH_TXT = OUT_DIR / "lgb_baseline_model.txt"
MODEL_PATH_JOBLIB = OUT_DIR / "lgb_baseline_model.joblib"
ENC_PATH = OUT_DIR / "label_encoders.joblib"

bst_final.save_model(str(MODEL_PATH_TXT))
joblib.dump(bst_final, str(MODEL_PATH_JOBLIB))

to_save = {'seg_te_map': seg_te_map}
if 'cluster_te_map' in globals():
    to_save['cluster_te_map'] = cluster_te_map
joblib.dump(to_save, str(ENC_PATH))

print("Saved model to:", MODEL_PATH_TXT, "and", MODEL_PATH_JOBLIB)
print("Saved encoders/maps to:", ENC_PATH)

Saved model to: ..\models\lgb_baseline_model.txt and ..\models\lgb_baseline_model.joblib
Saved encoders/maps to: ..\models\label_encoders.joblib


In [23]:
X_sample = X_test.sample(n=min(2000, len(X_test)), random_state=RND)

explainer = shap.TreeExplainer(bst_final)
svals = explainer.shap_values(X_sample)
shap_for_pos = svals[1] if isinstance(svals, (list,tuple)) else svals

plt.figure(figsize=(10,6))
try:
    shap.summary_plot(shap_for_pos, X_sample, show=False)
except Exception:
    shap.plots.beeswarm(shap.Explanation(values=shap_for_pos, data=X_sample, feature_names=list(X_sample.columns)))
out = FIG_DIR / "shap_summary_beeswarm.png"
plt.tight_layout()
plt.savefig(out, dpi=200, bbox_inches="tight")
plt.close()
print("Saved SHAP summary to:", out)

Saved SHAP summary to: ..\figures\shap_summary_beeswarm.png


In [24]:
feat = 'seg_te'
if feat in X_test.columns:
    X_dep = X_test.sample(n=min(5000, len(X_test)), random_state=RND)
    svals_dep = explainer.shap_values(X_dep)
    shap_vals_dep = svals_dep[1] if isinstance(svals_dep, (list,tuple)) else svals_dep

    plt.figure(figsize=(8,6))
    try:
        shap.dependence_plot(feat, shap_vals_dep, X_dep, show=False)
    except Exception:
        shap.plots.scatter(shap.Explanation(values=shap_vals_dep, data=X_dep, feature_names=list(X_dep.columns)), x=feat)
    out2 = FIG_DIR / "shap_dependence_seg_te.png"
    plt.tight_layout()
    plt.savefig(out2, dpi=200, bbox_inches="tight")
    plt.close()
    print("Saved SHAP dependence plot to:", out2)
else:
    print(feat, "not in features, skipping dependence plot.")

Saved SHAP dependence plot to: ..\figures\shap_dependence_seg_te.png


<Figure size 800x600 with 0 Axes>